In [2]:
%pip install transformers datasets evaluate accelerate

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Relevant model strings
human_class_longformer = "lenguist/longformer-coherence-1"
synth_class_longformer = "lenguist/longformer-coherence-synthetic-classifier"
synth_pref_longformer = "lenguist/longformer-coherence-synthetic-preference"

In [6]:
import pandas as pd
#choose file on which to eval
csv_file_path = "data/coherence/coherence_human_data.csv"
coherence_human_data_df = pd.read_csv(csv_file_path)

In [7]:
coherence_human_data_df

,premise,story1,story2,coherence_preference
0,The world ends in a cataclysmic event.,"One day, the world just ended.\n\n The sky ...",The world ended. The sky turned red and the gr...,story2
1,Brad realizes the error of his ways and comes ...,"Shannon drove down the road slowly, and her ha...","”\n\nBrad pulled Shannon close, wrapping his a...",story1
2,The group of survivors must face the enemy and...,"continued, “Thank you, we promise that we will...",Jake looked around and saw a group of building...,story2
3,Aimee Kincaid goes home after a long day at wo...,Aimee Kincaid came home to an empty apartment....,Aimee Kincaid came to the door of her apartmen...,story1
4,"Valerie Marx wakes up in a dark basement, boun...",Valerie Marx woke and found herself lying on t...,Valerie woke up sitting on the floor of a conc...,story1
...,...,...,...,...
546,Shannon Daniels loses her mother to cancer and...,"When Shannon’s mother died, it was her father ...",Shannon Daniels felt like a lost and lonely tw...,story2
547,Jenna wakes up in the hospital a week later to...,had leapt down from the fence and was now sitt...,<br><br>“How long have I been out?” Jenna aske...,story2
548,"Daisy's daughter, Lisa, becomes her primary ca...",and threw it on the ground. He glanced at a pi...,"<br><br>\t“No, it’s nothing,” I replied as I l...",story1
549,Natalie goes to the police and helps them appr...,"that month, Trevor Lockwood cleared his throat...",?<br><br>Magic</s>Covid restrictions ease in N...,story1


In [4]:
#This is for a model of a type classifier

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

def compare_texts(text1, text2, model_name="lenguist/longformer-coherence-1"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # Tokenize and compute logits for text1
    inputs_text1 = tokenizer(text1, return_tensors="pt", truncation=True, max_length=4096)
    with torch.no_grad():
        logits_text1 = model(**inputs_text1).logits

    # Tokenize and compute logits for text2
    inputs_text2 = tokenizer(text2, return_tensors="pt", truncation=True, max_length=4096)
    with torch.no_grad():
        logits_text2 = model(**inputs_text2).logits

    # Compare logits for label 1
    if logits_text1[0][1] > logits_text2[0][1]:
        return "text1"
    else:
        return "text2"

# Example usage
text1 = "Your first text here."
text2 = "Your second text here."
result = compare_texts(text1, text2)
print(result)


/Users/mbondarenko/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


text1


In [8]:
#This is for a model of a type preference

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

def compare_merged_texts(text1, text2, model_name="lenguist/longformer-coherence-1"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # Merge texts with a separator
    merged_text = f"{text1} [SEP] {text2}"

    # Tokenize and compute logits for the merged text
    inputs = tokenizer(merged_text, return_tensors="pt", truncation=True, max_length=4096)
    with torch.no_grad():
        logits = model(**inputs).logits

    # Compare logits and return the corresponding text
    if logits[0][0] > logits[0][1]:
        return "text1"
    else:
        return "text2"

# Example usage
text1 = "Your first text here."
text2 = "Your second text here."
result = compare_merged_texts(text1, text2)
print(result)


text1
